This script filters, cleans, and loads data from the EPI bulk dataset to be used with the dynamic SSPI.  

# Data Source

The 2018 SSPI originally used the Nitrogen Management Index sourced through the Environmental Performance Index (EPI) from their website, https://epi.yale.edu/.  The original values sourced for the SSPI were copied and pasted by hand from the EPI website.  The EPI website has since been updated, and the page from which the data were copied no longer exists.  The 2022 version of the page is available at https://epi.yale.edu/epi-results/2022/component/snm.

Fortunately, https://epi.yale.edu/downloads has a blurb which contains a link to https://sedac.ciesin.columbia.edu/data/set/epi-environmental-performance-index-2022/data-download for bulk data downloads.  The most recent bulk download is for 2022, and covers the years 1950 - 2022.

There are also previous years available for download.  It may be an interesting project to compare the EPI datasets over the years to see how they revise data and whether we think their methodology produces reliable results.  For now, we will focus on the 2022 dataset, which I have downloaded locally and uploaded to the Google Drive at https://drive.google.com/drive/folders/102Lg4NREiRhH_pzLabiVbkHvxRiaepDq?usp=sharing